<a href="https://colab.research.google.com/github/SanjanaMohan34/MGMT_467_Team4/blob/main/Unit2_Flights_Team4/Unit2_LilyLarson_BQML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#authenticate, import bigquery
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

In [2]:
#define variables
project_id = 'directed-bongo-471119-d1'
dataset = 'flights_data'
client = bigquery.Client(project=project_id)

# Regression

In [5]:
#linear regression model
sql_query = f"""
CREATE OR REPLACE MODEL
  `{project_id}.{dataset}.linear_reg_a2`
OPTIONS
  (model_type='LINEAR_REG',
   input_label_cols=['ArrDelay'])
AS
SELECT
  ArrDelay,
  DepDelay,
  Distance,
  Reporting_Airline,
  Origin,
  Dest,
  DayOfWeek
FROM
  `{project_id}.{dataset}.flights_raw`
"""
query_job = client.query(sql_query)
query_job.result()
print("Linear regression model trained successfully.")

Linear regression model trained successfully.


In [6]:
#evaluate and display results
import pandas as pd

sql_query = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL `directed-bongo-471119-d1.flights_data.linear_reg_a2`)
"""
query_job = client.query(sql_query)
eval_results = query_job.result()
eval_df = pd.DataFrame(eval_results.to_dataframe())
display(eval_df)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,415.524977,230351.481052,0.186425,425.359832,0.047356,0.047412


**Mean Absolute Error (MAE)**

Mean abolsute error is the average of the absolute values of the distances between each predicted point and the corresponding observed point. It measures a model's accuracy. Say we are predicting future stock price using historic data with time series cross validation. To calculate MAE, subtract the predicted value from the actual observed value for each day and take the absolute value of the difference. Sum these differences across each predicted point and take the average by dividing by the total number of predictions.

In [7]:
#ML.EXPLAIN_PREDICT
sql_query = f"""
SELECT *
FROM ML.EXPLAIN_PREDICT(MODEL `{project_id}.{dataset}.linear_reg_a2`,
  (
    SELECT
      CAST(NULL AS INT64) AS DepDelay,
      1000 AS Distance,
      'AA' AS Reporting_Airline,
      'JFK' AS Origin,
      32945 AS Dest,
      1 AS DayOfWeek
    UNION ALL
    SELECT
      CAST(NULL AS INT64) AS DepDelay,
      200 AS Distance,
      'UA' AS Reporting_Airline,
      'LAX' AS Origin,
      32457 AS Dest,
      5 AS DayOfWeek
  )
)
"""
query_job = client.query(sql_query)
explain_results = query_job.result()
explain_df = pd.DataFrame(explain_results.to_dataframe())
display(explain_df)

,predicted_ArrDelay,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,DepDelay,Distance,Reporting_Airline,Origin,Dest,DayOfWeek
0,1720.650721,"[{'feature': 'Origin', 'attribution': 5569.223...",-8553.471214,1720.650721,0.0,<NA>,1000,AA,JFK,32945,1
1,1520.849237,"[{'feature': 'Origin', 'attribution': 5567.182...",-8553.471214,1520.849237,0.0,<NA>,200,UA,LAX,32457,5


In [8]:
#display all values in top_feature_attributions
pd.set_option('display.max_colwidth', None)
print(explain_df['top_feature_attributions'])

0      [{'feature': 'Origin', 'attribution': 5569.223056799492}, {'feature': 'Reporting_Airline', 'attribution': 4475.289033514464}, {'feature': 'Distance', 'attribution': 225.7138972000376}, {'feature': 'DayOfWeek', 'attribution': 3.6628748125795005}, {'feature': 'Dest', 'attribution': 0.2330735589555628}]
1    [{'feature': 'Origin', 'attribution': 5567.18229670189}, {'feature': 'Reporting_Airline', 'attribution': 4486.9356954564055}, {'feature': 'Distance', 'attribution': 21.28873086015519}, {'feature': 'DayOfWeek', 'attribution': -1.2242199475757565}, {'feature': 'Dest', 'attribution': 0.13794807142198642}]
Name: top_feature_attributions, dtype: object


The top features were Origin, Reporting_Airline, Distance, DayofWeek, and Destination. The top feature, Origin, had an attribution of over 5560 for both hypothetical flights, indicating that the originating airport for a flight has a significant effect on the arrival delay time. Reporting_Airline had an attribution value of over 4470, indicating that it also has a substantial effect on the length of the arrival delay. Distance and DayOfWeek have inconsistent feature attribution values between the two flights, which makes them difficult to interpret. Destination has an attribution value of <0.25 for both flights, indicating that the flight's arrival location has a significantly lower affect on the quantity of arrival delay than other variables.

# Classification

In [9]:
#logistic regression model
sql_query = f"""
CREATE OR REPLACE MODEL
  `{project_id}.{dataset}.log_reg_a2`
OPTIONS
  (model_type='LOGISTIC_REG',
   input_label_cols=['Diverted'])
AS
SELECT
  Diverted,
  DepDelay,
  Distance,
  Reporting_Airline,
  Origin,
  Dest,
  DayOfWeek
FROM
  `{project_id}.{dataset}.flights_raw`
"""
query_job = client.query(sql_query)
query_job.result()
print("Logistic regression model trained successfully.")

Logistic regression model trained successfully.


In [10]:
#print results
sql_query = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL `{project_id}.{dataset}.log_reg_a2`)
"""
query_job = client.query(sql_query)
eval_results_log_reg = query_job.result()
eval_df_log_reg = pd.DataFrame(eval_results_log_reg.to_dataframe())
display(eval_df_log_reg)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.981408,0.0,0.087294,0.715269


In [11]:
#adjust threshold and print results
sql_query = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL `{project_id}.{dataset}.log_reg_a2`, (
    SELECT Diverted, Dest, Origin, DepDelay, Reporting_Airline, Distance, DayOfWeek
    FROM `directed-bongo-471119-d1.flights_data.flights_raw`),
  STRUCT (0.2 AS Threshold))
"""
query_job = client.query(sql_query)
eval_results_log_reg = query_job.result()
eval_df_log_reg = pd.DataFrame(eval_results_log_reg.to_dataframe())
display(eval_df_log_reg)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.144444,0.000456,0.982768,0.000909,0.082438,0.702353


In [12]:
#confusion matrix
sql_query = f"""
SELECT *
FROM
  ML.CONFUSION_MATRIX(MODEL `{project_id}.{dataset}.log_reg_a2`)
"""

query_job = client.query(sql_query)
confusion_matrix_results = query_job.result()
confusion_matrix_df = pd.DataFrame(confusion_matrix_results.to_dataframe())
display(confusion_matrix_df)

,expected_label,_0,_1
0,0,9871,0
1,1,187,0


**Threshold Justification**

It seems that very few flights in our dataset are diverted, causing our model to want to predict all flights as undiverted, lending a high accuracy, but minimal precision, recall, and f1 scores. Lowering the threshold increases the chances that we will predict a diverted flight, which is valuable to us. We would much rather overpredicit diverted flights so that we can be prepared and limit customer expectations than fail to know when a flight will be diverted. Lowering the threshold to 0.2 only slightly decreases the area under the curve and increaeses accuracy, precision, recall, and f1 score (altough the latter 3 still remain very small).

#Feature Engineering

In [13]:
#transforme features, create view
sql_create_view = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset}.flights_transformed_view` AS
SELECT
  Diverted,
  Distance,
  Reporting_Airline,
  Origin,
  Dest,
  DayOfWeek,
  DepDelay,
  CONCAT(Origin, '-', Dest) AS route,
  CASE
    WHEN DepDelay < 0 THEN 'Early'
    WHEN DepDelay = 0 THEN 'On-time'
    WHEN DepDelay > 0 AND DepDelay <= 15 THEN 'Minor'
    WHEN DepDelay > 15 AND DepDelay <= 60 THEN 'Moderate'
    ELSE 'Major'
  END AS DepDelay_bucket
FROM
  `{project_id}.{dataset}.flights_raw`
"""

query_job_view = client.query(sql_create_view)
query_job_view.result()
print("Transformed view 'flights_transformed_view' created successfully.")

Transformed view 'flights_transformed_view' created successfully.


In [14]:
#create model
sql_create_model_from_view = f"""
CREATE OR REPLACE MODEL
  `{project_id}.{dataset}.log_reg_a2_transformed`
OPTIONS
  (model_type='LOGISTIC_REG',
   input_label_cols=['Diverted'])
AS
SELECT
  Diverted,
  Distance,
  Reporting_Airline,
  Origin,
  Dest,
  DayOfWeek,
  route,
  DepDelay_bucket
FROM
  `{project_id}.{dataset}.flights_transformed_view`
"""

query_job_model = client.query(sql_create_model_from_view)
query_job_model.result()
print("Logistic regression model trained successfully using the transformed view.")

Logistic regression model trained successfully using the transformed view.


In [15]:
#print results
sql_query = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL `{project_id}.{dataset}.log_reg_a2_transformed`)
"""
query_job = client.query(sql_query)
eval_results_log_reg = query_job.result()
eval_df_log_reg = pd.DataFrame(eval_results_log_reg.to_dataframe())
display(eval_df_log_reg)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.982876,0.0,0.083952,0.658759


In [16]:
#compare metrics side-by-side
import pandas as pd

sql_query_original = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL `{project_id}.{dataset}.log_reg_a2`)
"""
query_job_original = client.query(sql_query_original)
eval_df_original = pd.DataFrame(query_job_original.result().to_dataframe())
eval_df_original['Model'] = 'log_reg_a2'

sql_query_transformed = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL `{project_id}.{dataset}.log_reg_a2_transformed`)
"""
query_job_transformed = client.query(sql_query_transformed)
eval_df_transformed = pd.DataFrame(query_job_transformed.result().to_dataframe())
eval_df_transformed['Model'] = 'log_reg_a2_transformed'

comparison_df = pd.concat([eval_df_original, eval_df_transformed])
comparison_df = comparison_df.set_index('Model')
display(comparison_df)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
Model,,,,,,
log_reg_a2,0.0,0.0,0.981408,0.0,0.087294,0.715269
log_reg_a2_transformed,0.0,0.0,0.982876,0.0,0.083952,0.658759


**Results Comparison**

Overall, performance is slightly worse than the model without transformed features. Accuracy  and log loss both improved, but only by insignificant amounts. However, AUC dropped by around 0.04.

**Precision vs. Recall**

In this scenario, we will prioritize recall over precision. Precision measures the accuracy of positive predictions (TP/TP+FP) and is punished by a high percentage of false positive predictions. False positives for this model would be flights that are predicted to be delayed, but depart on time. Recall measures the model's success in finding the true positive cases (TP/TP+FN). More false negative predictions lower the value of this metric. A false negative prediction in this setting would be a flight predicted to depart on time that is delayed in reality. These are more important to us because delayed flights cause both operational headaches and upset customers. Most ticket holders arrive to the airport in plenty of time to catch their flights and would likely appreciate a flight predicted to be delayed that actually departs on time (false positive). The negative reaction to delayed flights indicates that we should prioritize correctly predicting all delayed flights.  

# Cost and Scale


**Limit**

The flights_raw table is 3 months worth of data and compromises 1,658,259 rows. While this is a large amount, when selecting a specific subset of attibutes in queries, the information is managable and the results are still delivered quickly. For queries asking for a return of values, I would recommend limiting output to between 100-1000 rows. We can continue to build and test models on this subset of the data before deloying on the entire collection of available or relevant data. A query using a LIMIT approach could look like:

SELECT *

FROM `flights_data.flights_raw`

WHERE Diverted = 1

LIMIT 100

This will return a list of 100 flights that were diverted.

**Final Run Plan - Sample vs. Full Table Trade-off**

After using sample data for development and testing, we will deploy our model on the full dataset. This will give the model access to all available data, allowing actionable insights to be determined. The model should still be monitored, as exposure to rare cases or new trends not represented in sample data may hinder performance. In cases of poor performance, we will aim to isolate the source of the irregularity and include it in a new set of training data to fine-tune the model's performance.